In [1]:
import os
import gradio as gr
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

In [2]:
os.environ['OPENAI_API_KEY'] = '' 

In [3]:
loader = TextLoader('dados.txt')
documents = loader.load()

In [4]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = text_splitter.split_documents(documents)

In [5]:
embeddings = OpenAIEmbeddings()
vector_store = FAISS.from_documents(docs, embeddings)
retriever = vector_store.as_retriever()

In [6]:
llm = ChatOpenAI(model_name='gpt-3.5-turbo')
qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever, chain_type="stuff")

In [7]:
def chat_rag(history, message):
    if history is None:
        history = []
    
    response = qa_chain.invoke(message)['result']
    
    history.append({'role': 'user', 'content': message})
    history.append({'role': 'assistant', 'content': response})
    
    return history, ''

In [8]:
with gr.Blocks() as demo:
    gr.Markdown('# Chat RAG - Baseado no seu arquivo de texto')
    
    chatbot = gr.Chatbot(type='messages')
    msg = gr.Textbox(label='Digite sua pergunta')
    send_btn = gr.Button('Enviar')
    
    send_btn.click(chat_rag, [chatbot, msg], [chatbot, msg])

In [9]:
demo.launch()

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
